In [ ]:
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.utils import np_utils, to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD
from keras.datasets import cifar10
from keras import regularizers
from keras.callbacks import LearningRateScheduler
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
tf.logging.set_verbosity(tf.logging.ERROR)

In [ ]:
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 75:
        lrate = 0.0005
    if epoch > 100:
        lrate = 0.0003
    return lrate

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [ ]:
# prep the data by normalizing
# and doing a one-hot encoding for the labels

num_classes = 10
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std)
x_test = (x_test-mean)/(std)

y_train = to_categorical(y_train,num_classes)
y_test = to_categorical(y_test,num_classes)

In [ ]:
weight_decay = 1e-4
model = Sequential()
model.add(Conv2D(32,(3,3), activation='relu', padding='same', \
                 kernel_regularizer=regularizers.l2(weight_decay), \
                 input_shape=x_train.shape[1:]))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3), activation='relu', padding='same', \
                 kernel_regularizer=regularizers.l2(weight_decay)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64,(3,3), activation='relu', padding='same', \
                 kernel_regularizer=regularizers.l2(weight_decay)))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3), activation='relu', padding='same', \
                 kernel_regularizer=regularizers.l2(weight_decay)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(128,(3,3), activation='relu', padding='same', \
                 kernel_regularizer=regularizers.l2(weight_decay)))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3), activation='relu', padding='same', \
                 kernel_regularizer=regularizers.l2(weight_decay)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(num_classes,activation='softmax'))
model.summary()

In [ ]:
# Data Augmentation - makes copies of the samples in training data with random mods

datagen = ImageDataGenerator(
            rotation_range= 15,
            width_shift_range=0.1,
            height_shift_range=0.1,
            horizontal_flip=True,)
datagen.fit(x_train)

In [ ]:
def plot_results(history):
    plt.subplot(221)
    plt.title('Entropy Loss')
    plt.plot(history.history['loss'], color='blue', label='train')
    plt.plot(history.history['val_loss'], color='orange', label='test')
    plt.xlabel("Num of Epochs")
    plt.ylabel("Loss")
    plt.title("Training vs Validation Loss")
    plt.subplot(222)
    plt.title("Classifier Accuracy")
    plt.plot(history.history['acc'], color='blue', label='train')
    plt.plot(history.history['val_acc'], color='orange', label='test')
    plt.xlabel("Num of Epochs")
    plt.ylabel("Accuracy")
    plt.title("Training vs Validation Accuracy")
    plt.legend(['train','validation'])
    plt.tight_layout()
    plt.show()

In [ ]:
# Train the model
batch_size=64
opt_sgd = SGD(lr=0.001, momentum=0.9)
model.compile(optimizer=opt_sgd, loss='categorical_crossentropy', metrics=['accuracy'])
data = model.fit_generator(datagen.flow(x_train,y_train,batch_size=batch_size), \
                    steps_per_epoch=x_train.shape[0]/batch_size, \
                    epochs=125, verbose=1, validation_data=(x_test,y_test), \
                    callbacks=[LearningRateScheduler(lr_schedule)])
scores = model.evaluate(x_test, y_test, batch_size=128,verbose=1)
print('\nTest Results: %.3f loss: %.3f' % (scores[1] * 100, scores[0]))
plot_results(data)